In [41]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from data import *
from _version import __version__
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:

from _version import __version__
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()
    # delete incomplete experiments
    dbHandler.delete_incomplete_experiments()
print("Current code version, ", __version__)

Current code version,  11


In [62]:
# get the completed experiments with the current code version ordered by date
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed
9,6302a2e9-7b4e-4f52-8ed2-7645696d801c,Default Experiment,38.930573,11,2023-03-30 10:56:36,10,10,"[0.2, 1.0, 2.0, 5.0]",[0],[1],0.0,1000,"[""gd""]",1
8,15bf551f-ae35-4b95-bb90-972c62345dda,Default Experiment,164.196687,11,2023-03-30 10:52:48,10,10,"[0.2, 1.0, 2.0, 5.0]","[0.0, 0.01, 0.2, 1.0]",[1],2.0,1000,"[""gd""]",1


In [63]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

6302a2e9-7b4e-4f52-8ed2-7645696d801c


In [64]:
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
gd = erm[erm["chosen_minimizer"] == "gd"]
# set alpha, epsilon, tau, lam as index in gd and state_evolution
gd = gd.set_index(["alpha","epsilon","tau","lam"])
state_evolution = state_evolution.set_index(["alpha","epsilon","tau","lam"])
# from state_evolution keep generalization_error, sigma, q, m, cosb
state_evolution = state_evolution[["generalization_error","sigma","q","m","cosb"]]
# add a suffix _state_evolution to the columns
state_evolution.columns = [col+"_state_evolution" for col in state_evolution.columns]
# from gd keep generalization_error_erm	generalization_error_overlap Q	rho	m	cosb
gd = gd[["generalization_error_erm","generalization_error_overlap","Q","rho","m","cosb"]]
# add a suffix _erm to the columns
gd.columns = [col+"_erm" for col in gd.columns]
# join the two dataframes
df = state_evolution.join(gd)
# reorder the columns
df = df[['generalization_error_state_evolution', 'generalization_error_erm_erm', 'generalization_error_overlap_erm', 'sigma_state_evolution', 'q_state_evolution','Q_erm','m_state_evolution','m_erm','cosb_state_evolution','cosb_erm', 'rho_erm']]
# average all quantities over the index and obtain also the standard deviation
df = df.groupby(level=[0,1,2,3]).agg([np.mean,np.std])
# display all columns
pd.set_option('display.max_columns', None)
df

generalization_error_state_evolution       \
                                                      mean  std   
alpha epsilon tau lam                                             
0.2   0.0     0.0 1.0                             0.496038  0.0   
1.0   0.0     0.0 1.0                             0.481933  0.0   
2.0   0.0     0.0 1.0                             0.467214  0.0   
5.0   0.0     0.0 1.0                             0.433827  0.0   

                      generalization_error_erm_erm            \
                                              mean       std   
alpha epsilon tau lam                                          
0.2   0.0     0.0 1.0                     0.392406  0.006081   
1.0   0.0     0.0 1.0                     0.270574  0.004438   
2.0   0.0     0.0 1.0                     0.204016  0.004133   
5.0   0.0     0.0 1.0                     0.123530  0.002665   

                      generalization_error_overlap_erm            \
                                                  mean       std   
alpha epsilon tau lam                                              
0.2   0.0     0.0 1.0                         0.392089  0.005123   
1.0   0.0     0.0 1.0                         0.270250  0.004355   
2.0   0.0     0.0 1.0                         0.204388  0.004400   
5.0   0.0     0.0 1.0                         0.123584  0.002824   

                      sigma_state_evolution      q_state_evolution       \
                                       mean  std              mean  std   
alpha epsilon tau lam                                                     
0.2   0.0     0.0 1.0              0.961886  0.0          0.030428  0.0   
1.0   0.0     0.0 1.0              0.831473  0.0          0.122507  0.0   
2.0   0.0     0.0 1.0              0.706875  0.0          0.190305  0.0   
5.0   0.0     0.0 1.0              0.479589  0.0          0.254094  0.0   

                          Q_erm           m_state_evolution          m_erm  \
                           mean       std              mean  std      mean   
alpha epsilon tau lam                                                        
0.2   0.0     0.0 1.0  0.031645  0.001790          0.012398  0.0  0.059182   
1.0   0.0     0.0 1.0  0.179398  0.004673          0.055407  0.0  0.279227   
2.0   0.0     0.0 1.0  0.381019  0.003812          0.097389  0.0  0.494741   
5.0   0.0     0.0 1.0  1.024422  0.003728          0.176787  0.0  0.935645   

                                cosb_state_evolution       cosb_erm            \
                            std                 mean  std      mean       std   
alpha epsilon tau lam                                                           
0.2   0.0     0.0 1.0  0.003699             0.012446  0.0  0.332513  0.015130   
1.0   0.0     0.0 1.0  0.009796             0.056728  0.0  0.660662  0.010245   
2.0   0.0     0.0 1.0  0.010771             0.102818  0.0  0.800761  0.008356   
5.0   0.0     0.0 1.0  0.015140             0.206394  0.0  0.925537  0.003386   

                        rho_erm            
                           mean       std  
alpha epsilon tau lam                      
0.2   0.0     0.0 1.0  1.001558  0.038460  
1.0   0.0     0.0 1.0  0.995799  0.038403  
2.0   0.0     0.0 1.0  1.002022  0.031434  
5.0   0.0     0.0 1.0  0.997910  0.033323

In [ ]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)